This notebook will look at developing a probabilistic binary threshold filter. The idea is to provide a binary threshold with a degree of randomness, where the magnitude of the input value influences how probable a 1 or 0 is.

The filter will be designed to work on numpy arrays of varying sizes.

## Random Numbers

We need a set of random numbers to compare our input to. The range will depend on our input. If our input is 8-bit, our highest value is 256, 16-bit, 2^16 etc.

So we can have a variable bit_size that indicates the bit size of the input and our highest value is 2^bit_size.

In [3]:
import numpy as np
np.version.version

'1.16.2'

In [7]:
from packaging import version
version.parse(np.version.version)

<Version('1.16.2')>

In [11]:
# This only works for numpy v.1.17+

# These variables will be set based on the input properties
# Set bit_size
bit_size = 8
# Set size
input_size = (5, 5)

# Set highest integer value
high = 2**bit_size

# Code to account for different numpy versions
if version.parse(np.version.version) < version.parse("1.17.0"):
    rand_ints = np.random.randint(high, size=input_size)
else:
    # Setup a random number generator
    rng = np.random.default_rng()
    rand_ints = rng.integers(high, size=input_size)
print(rand_ints)

[[ 18 210  69  95 241]
 [250 212  48 173 119]
 [142  12 162 105  43]
 [ 16 100  86 248 171]
 [ 10  57 178 108  28]]


We can also set the data type based on what category bit_size falls into.

In [12]:
def get_rand_ints(bit_size, input_size):
    """Output a set of random integers of size input_size
    of the indicated bit size."""
    # Set highest integer value
    high = 2**bit_size

    # Code to account for different numpy versions
    if version.parse(np.version.version) < version.parse("1.17.0"):
        rand_ints = np.random.randint(high, size=input_size)
    else:
        # Setup a random number generator
        rng = np.random.default_rng()
        rand_ints = rng.integers(high, size=input_size)
    return rand_ints

In [13]:
get_rand_ints(8, (5,5))

array([[ 88, 176,  60,  65, 223],
       [252,  25, 154, 137,  81],
       [159,  38, 182, 241,  14],
       [219, 133,  88,  89, 119],
       [243, 160, 184,  23, 142]])

## Comparison

Code to compare an input and output binary values.

In [17]:
input_values = get_rand_ints(8, (5,5))
rand_ints = get_rand_ints(8, (5,5))
print(input_values, '\n', rand_ints, '\n', input_values>rand_ints)

[[ 56 183 225  88   1]
 [100  64 118  85 195]
 [163 252 165  25  16]
 [ 98 111 214  72   4]
 [183  92 125  10 137]] 
 [[215 237 192 165   6]
 [104  47 252  60  63]
 [103 204 227 174 128]
 [147  36 207 204  26]
 [209 225 211 222  57]] 
 [[False False  True False False]
 [False  True False  True  True]
 [ True  True False False False]
 [False  True  True False False]
 [False False False False  True]]


In [19]:
input_values = get_rand_ints(8, (5,5))
rand_ints = get_rand_ints(8, (5,5))
print(input_values, '\n', rand_ints, '\n', np.where(input_values>rand_ints, 1, 0))

[[207  25  32  95 141]
 [226  85   4   4 227]
 [157 136 105  94  68]
 [177 212 143  47 149]
 [109  33  51  74 177]] 
 [[184 110 107  54 137]
 [173 203 157   3 221]
 [ 10  29  21  14   2]
 [150   5 232  38 185]
 [177 157  29 222 237]] 
 [[1 0 0 1 1]
 [1 0 0 1 1]
 [1 1 1 1 1]
 [1 1 0 1 0]
 [0 0 1 0 0]]


In [21]:
print(input_values.dtype, input_values.dtype.itemsize)

int64 8


In [23]:
new_array = np.array([100, 100], dtype=np.int8)
print(new_array.dtype, new_array.dtype.itemsize)

int8 1


Ah itemsize is in bytes.

In [33]:
def pb_threshold(input_values):
    """Apply a probablistic binary threshold to the input_values"""
    input_size = input_values.size
    data_type = input_values.dtype
    bit_size = data_type.itemsize*8
    rand_ints = get_rand_ints(bit_size, input_size)
    print(input_values, '\n', rand_ints, '\n', np.where(input_values>rand_ints, 1, 0))
    binary_values = np.where(input_values>rand_ints, 1, 0)
    return binary_values

In [40]:
pb_threshold(np.array([155], dtype=np.uint8))

[155] 
 [74] 
 [1]


array([1])

Ah we need to beware of signed and unsigned values. Here we assume all values are unsigned.